PRACTICE ASSIGNMENT 1

In [ ]:
import pandas as pd
import matplotlib
import numpy as np

In [ ]:
df = pd.read_csv("/content/Preprocessing1.csv")
df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month,WeekDay,Day
0,Jet Airways,Delhi,Banglore,DEL → BOM → COK,20:00,10-06-2026 04:25,26h 35m,1 stop,In-flight meal not included,14924,6,Thursday,6.0
1,Jet Airways,Delhi,Cochin,DEL → BOM → COK,16:00,10-06-2026 19:00,27h,1 stop,In-flight meal not included,10577,6,Sunday,9.0
2,Jet Airways,Mumbai,Hyderabad,BOM → HYD,19:35,21:05,1h 30m,non-stop,No info,5678,3,Friday,15.0
3,Multiple carriers,Delhi,Banglore,DEL → BOM → COK,18:55,16-06-2026 01:30,15h 10m,1 stop,In-flight meal not included,7408,5,Monday,6.0
4,Air India,Delhi,Cochin,DEL → COK,17:10,17:55,8h 20m,non-stop,No info,6724,6,Monday,24.0


In [ ]:
# -------------------------------
# 1️⃣ Average flight ticket price
# -------------------------------
avg_price = df["Price"].mean()
print("Average Price:", round(avg_price, 2))

Average Price: 9027.9


In [ ]:
# ------------------------------------------------
# 2️⃣ Month with highest number of flights (mode)
# ------------------------------------------------
month_counts = df["Month"].value_counts()
print(month_counts)
highest_month = month_counts.idxmax()
print("Month with highest flights:", highest_month)

Month
5    3092
6    3044
3    2388
4     926
Name: count, dtype: int64
Month with highest flights: 5


In [ ]:
# ----------------------------------------------------------------
# 3️⃣ Compare average ticket price on weekends vs weekdays
# ----------------------------------------------------------------
weekend_days = ["Saturday", "Sunday"]

weekend_avg = df[df["WeekDay"].isin(weekend_days)]["Price"].mean()
weekday_avg = df[~df["WeekDay"].isin(weekend_days)]["Price"].mean()

if weekend_avg > weekday_avg:
    print("Higher average price on: Weekends")
else:
    print("Higher average price on: Weekdays")

Higher average price on: Weekends


In [ ]:
# ----------------------------------------------------------------
# 4️⃣ Standardize 'No Info' → 'No info'
# ----------------------------------------------------------------
df["Additional_Info"] = df["Additional_Info"].replace("No Info", "No info")

indigo_noinfo_count = df[
    (df["Airline"] == "IndiGo") &
    (df["Additional_Info"] == "No info")
].shape[0]

print("IndiGo + No info flights:", indigo_noinfo_count)

IndiGo + No info flights: 1650


In [ ]:
# ----------------------------------------------------------------
# 5️⃣ Convert Duration → seconds & compute average
# ----------------------------------------------------------------
def duration_to_seconds(duration):
    """
    Converts strings like '2h 50m' → seconds
    """
    hours = 0
    minutes = 0

    if pd.isna(duration):
        return np.nan

    parts = duration.split()

    for part in parts:
        if "h" in part:
            hours = int(part.replace("h", ""))
        elif "m" in part:
            minutes = int(part.replace("m", ""))

    return hours * 3600 + minutes * 60


df["Duration_seconds"] = df["Duration"].apply(duration_to_seconds)

avg_duration_sec = df["Duration_seconds"].mean()
print("Average Duration (seconds):", round(avg_duration_sec, 2))


Average Duration (seconds): 38957.94


In [ ]:
# ----------------------------------------------------------------
# 6️⃣ Extract hour from Dep_Time & Arrival_Time
# ----------------------------------------------------------------
import pandas as pd
import numpy as np

# Convert Dep_Time and Arrival_Time to datetime
df["Dep_Time"] = pd.to_datetime(df["Dep_Time"], errors="coerce")
df["Arrival_Time"] = pd.to_datetime(df["Arrival_Time"], errors="coerce")

# Extract hour safely
df["Dep_Hour"] = df["Dep_Time"].dt.hour
df["Arrival_Hour"] = df["Arrival_Time"].dt.hour

# ----------------------------------------------------------------
# Categorize into Morning / Afternoon / Evening / Night
# ----------------------------------------------------------------
def categorize_time(hour):
    if pd.isna(hour):
        return np.nan

    if 5 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 17:
        return "Afternoon"
    elif 17 <= hour < 20:
        return "Evening"
    else:
        return "Night"
df["Dep_Category"] = df["Dep_Hour"].apply(categorize_time)
df["Arrival_Category"] = df["Arrival_Hour"].apply(categorize_time)

# ----------------------------------------------------------------
# Count flights: Morning departure → Evening arrival
# ----------------------------------------------------------------
morning_to_evening = df[
    (df["Dep_Category"] == "Morning") &
    (df["Arrival_Category"] == "Evening")
].shape[0]

print("Morning → Evening flights:", morning_to_evening)



Morning → Evening flights: 76


/tmp/ipython-input-504654675.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Dep_Time"] = pd.to_datetime(df["Dep_Time"], errors="coerce")


In [ ]:
# ----------------------------------------------------------------
# 7️⃣ Encode WeekDay → Weekend = 1, Weekday = 0
# ----------------------------------------------------------------
df["WeekDay_Encoded"] = df["WeekDay"].apply(
    lambda x: 1 if x in ["Saturday", "Sunday"] else 0
)

mode_day = df["WeekDay_Encoded"].mode()[0]

if mode_day == 1:
    print("Most frequent WeekDay: Weekends")
else:
    print("Most frequent WeekDay: Weekdays")

Most frequent WeekDay: Weekdays
